In [29]:
from newspaper import Article
import pandas as pd
import json


In [7]:
def extract_article(url):
    article = Article(url)
    article.download()
    article.parse()
    return article.title, article.text

In [8]:
article_template = {
    "title": "",
    "text": ""
}

In [9]:
import unicodedata
import ftfy


def clean_text(text):
    if not text or not text.strip():  # Check for None, empty, or whitespace-only input
        return text  # Return as is (or return an empty string if preferred)
    
    # Fix text encoding issues
    text = ftfy.fix_text(text)
    
    # Normalize unicode characters (e.g., fix apostrophes, ellipses, etc.)
    text = unicodedata.normalize("NFC", text)
    
    return text

In [10]:

# data = pd.read_csv("mini-data.csv")
data = pd.read_csv("data/train-data.csv")

data_v1 = data[['url', 'title', 'content', 'lang', 'post_type']].copy()

print(f"Data len: {len(data_v1)}")


Data len: 660


2 removed.

In [ ]:
exception_counter = 0
for index, entity in data_v1.iterrows():
    try:
        title, content = extract_article(entity['url'])
        data_v1.at[index, "extracted_title"] = clean_text(title)
        data_v1.at[index, "extracted_content"] = clean_text(content)
        data_v1.at[index, "title"] = clean_text(data_v1.at[index, "title"])
        data_v1.at[index, "content"] = clean_text(data_v1.at[index, "content"])
        data_v1.at[index, "post_type"] = clean_text(data_v1.at[index, "post_type"])
        data_v1.at[index, "content_extracted"] = True
        data_v1.at[index, "error"] = ""
    except Exception as e:
        exception_counter += 1
        print(f"Error processing row {index}: {e}")
        print(f"URL: {entity['url']}")
        data_v1.at[index, "content_extracted"] = False
        data_v1.at[index, "error"] = f"{e}"
        print(f'----------------------------------------------------------------')

print(f"Number of exceptions: {exception_counter}/{len(data_v1)}")
print(f"% of exceptions: {exception_counter*100/len(data_v1)} %")
data_v1.to_json(f"processed_data_658.json", orient="records", lines=True, force_ascii=False)

In [30]:
# count words

content_counter = 0
extended_content_counter = 0
data_v2 = pd.read_json("data/processed_data_658.json", orient="records", lines=True)

num_con = 0
num_ext_con = 0

train_data = []

for index, entity in data_v2.iterrows():
    source_of_extended_content = ''
    is_extracted = data_v2.at[index, "content_extracted"]
    content = data_v2.at[index, "content"]
    extracted_content = data_v2.at[index, "extracted_content"]
    print("--------------------------------------------------------------------------------------------------------------------------")
    print(f"Index: {index}")
    if is_extracted and len(extracted_content) > len(content):
        extended_content_counter += len(extracted_content.split())
        source_of_extended_content = 'content_extracted'
        num_ext_con += 1
        train_data.append(extracted_content)
    else:
        extended_content_counter += len(content.split())
        source_of_extended_content = 'content'
        num_con += 1
        train_data.append(content)
        
    print(f"source_of_extended_content: {source_of_extended_content}")
    print("---------------------------------")
    print(f"extracted_content: {extracted_content}")
    print("---------------------------------")
    print(f"content: {content}")
    print("---------------------------------")
    print(f"is_extracted: {is_extracted}")
    content_counter += len(content.split())


print("--------------------------------------------------------------------------------------------------------------------------")
print("--------------------------------------------------------------------------------------------------------------------------")

print(f"Content words counter: {content_counter}")
print(f"num_con: {num_con}")
print(f"Extended content words counter: {extended_content_counter}")
print(f"num_ext_con: {num_ext_con}")

print(f"data_v2: {len(data_v2)}")
print(f"train_data: {len(train_data)}")


# Save to a JSON file
with open("data/training_data_658.json", "w", encoding="utf-8") as f:
    json.dump(train_data, f, ensure_ascii=False, indent=4)

--------------------------------------------------------------------------------------------------------------------------
Index: 0
source_of_extended_content: content
---------------------------------
extracted_content: None
---------------------------------
content: La réalisatrice helvético-péruvienne Klaudia Reynicke remporte le prix du public du Festival de Locarno. Entre monde rêvé et réalisme, son film 'Reinas' (Les reines) est une chronique familiale qui se déroule dans les années 90 à Lima.

Le film a con...
---------------------------------
is_extracted: False
--------------------------------------------------------------------------------------------------------------------------
Index: 1
source_of_extended_content: content
---------------------------------
extracted_content: None
---------------------------------
content: @GoeyvaertsJrme1 @DansLaMusette Belle saison de merde pour lui c'est pas le tour de Romandie je pense qui va le rendre heureux
---------------------------